In [1]:
import pandas as pd

# Load Data

## Report Text Data

In [119]:
df_text = pd.read_csv("data/[DEIDENTIFIED]OACIS_RadiologyReport_20241024.csv", usecols=["MRN", "ReportDate", "ServDescription", "ReportText"])
df_text.rename(columns={"MRN": "pt_shsc_id"}, inplace=True)
df_text["ReportDate"] = pd.to_datetime(df_text["ReportDate"])
df_text["ReportDate"] = df_text["ReportDate"].dt.strftime('%Y-%m-%d')

In [120]:
print(f"number of radiology reports: {df_text.shape}")
print(f"number of unique patient IDs: {df_text['pt_shsc_id'].unique().shape[0]}")

number of radiology reports: (29498, 4)
number of unique patient IDs: 1007


In [121]:
df_text

,pt_shsc_id,ServDescription,ReportDate,ReportText
0,SHSC-2Z1IY-LHGS0-6OLDN-SGN7V6QK09-QUXFB-C9QAG-...,BONE DENSITOMETRY MULT. SITE NM,1998-11-20,BONE DENSITOMETRY\r\nNo previous available for...
1,SHSC-2QDGI-JO60U-D33TD-DRTI2BV1RT-TZG89-7D46O-...,MYELOGRAM - LUMBAR,1999-03-04,CT MYELOGRAM LUMBAR SPINE\r\nNo previous for c...
2,SHSC-36H29-I47NO-NWSSS-QY16H8CRPA-9CMNJ-YS3BR-...,CHEST PA AND LAT (ROUTINE),1999-03-08,CHEST PA AND LATERAL\r\nPLEURAL PARENCHYMAL SC...
3,SHSC-2QDGI-JO60U-D33TD-DRTI2BV1RT-TZG89-7D46O-...,K.U.B.,2000-02-15,K.U.B.\r\n\r\nNo films are available for compa...
4,SHSC-1V4LU-PXF4M-WRME6-FMN0UK5S6N-QOBI8-NZRBM-...,Kidney + Bladder US,2000-04-27,KIDNEY & BLADDER ULTRASOUND:\r\n\r\nCOMPARISON...
...,...,...,...,...
29493,SHSC-1VI24-3RLMJ-3ZNTQ-BNY6TIQ7M1-6X9GJ-3DWIK-...,CT Chest,2023-06-22,"CT CHEST\r\n\r\n\r\nProvided history:NONE, met..."
29494,SHSC-30C4O-B9B1Y-5U42C-KPXDLBCPEC-8TJOT-Q6DQF-...,Abdomen + Pelvis CT with oral C+,2023-07-29,CT ABDOMEN AND PELVIS\r\n\r\n\r\nCLINICAL HIST...
29495,SHSC-1ZDWU-QIZZJ-BC7Q5-0S8K1XR7XA-OPNA2-WAF8Q-...,Bone Scan WB+ Spect,2024-03-18,TECHNETIUM-99m MDP BONE SCAN: WHOLE BODY\r\n\r...
29496,SHSC-D66G3-ZOD02-JNKZ3-NYLF7UPC9M-5KNWS-ICJ5L-...,Abdomen + Pelvis CT with oral C+,2024-02-27,CT ABDOMEN AND PELVIS\r\n\r\n\r\nVolumetric im...


## Report Labels

In [242]:
# `report_data` stores patient fracture labels (fracture, no fracture) and other results.
df_labels = pd.read_csv("data/Osteosarc_Rad_Report_Data_Oct_7.csv", usecols=[
	"pt_shsc_id",
	"imaging_date",
	'image_ct___1', 
	'image_ct___2', 
	'image_ct___3',
])

df_labels.rename(columns={
	"image_ct___1": "fracture",
	"image_ct___2": "metastases",
	"image_ct___3": "healthy"
}, inplace=True)

df_labels["fracture_or_metastases"] = df_labels["fracture"] + df_labels["metastases"]
df_labels["fracture_or_metastases"] = df_labels["fracture_or_metastases"].apply(lambda x: min(x["fracture_or_metastases"], 1))
df_labels.drop(columns=["fracture", "metastases"], inplace=True)

In [243]:
df_labels = df_labels[~df_labels["healthy"].isna()]

In [248]:
print(
	f"count of metastases or fracture detected: {len(df_labels[df_labels['fracture_or_metastases'] >= 1])}/{len(df_labels)}"
)
print(
	f"count of healthy: {len(df_labels[df_labels['healthy'] == 1])}/{len(df_labels)}"
)
print(
	f"count of no records: {len(df_labels[(df_labels['healthy'] == 0) & (df_labels['fracture_or_metastases'] == 0)])}/{len(df_labels)}"
)

count of metastases or fracture detected: 778/1269
count of healthy: 365/1269
count of no records: 126/1269


# Matching